In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, \
TensorDataset, random_split
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


In [2]:
# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

In [3]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [4]:
# Split the data: 60% training, 20% validation, 20% testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [5]:
# Standardize the training data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Apply the same transformation to the validation and test data
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [6]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [7]:
# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [8]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [9]:
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 32)
        self.fc3 = nn.Linear(32, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [17]:
# Instantiate the model
model = IrisNet()

# Define the loss function and optimizer with L2 regularization
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), 
                       lr=0.001, 
                       weight_decay=0.01)


In [21]:
# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        # This loop iterates over the 
        # mini-batches of data provided 
        # by train_loader. Each mini-batch 
        # contains a small subset of the 
        # training data.
        optimizer.zero_grad()

        # The forward pass is performed here. 
        # The model processes the inputs and 
        # produces outputs (predictions).        
        outputs = model(inputs)

        # The loss function (defined by criterion) 
        # calculates the difference between the 
        # model's predictions (outputs) and the
        # true labels (labels). The result is
        # the loss, which indicates how well
        # the model's predictions match 
        # the actual values.
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            # A forward pass is performed on 
            # the validation data to generate predictions.
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # This line finds the class with the 
            # highest predicted probability for 
            # each sample. The torch.max function
            # returns both the maximum value and 
            # the index of the maximum value along
            # the specified dimension (in this case,
            # the class index is returned).
            # Second Parameter (1): This specifies 
            # the dimension along which to look for
            # the maximum value. 0 refers to the 
            # batch dimension. 1 refers to the 
            # class dimension
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            # (predicted == labels).sum() is a tensor. 
            # dot item gives the scalar value.
            correct += (predicted == labels).sum().item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {running_loss/len(train_loader):.4f}, '
          f'Val Loss: {val_loss/len(val_loader):.4f}, '
          f'Val Accuracy: {100 * correct / total:.2f}%')

Epoch [1/20], Train Loss: 0.2766, Val Loss: 0.1926, Val Accuracy: 90.00%
Epoch [2/20], Train Loss: 0.2656, Val Loss: 0.1867, Val Accuracy: 93.33%
Epoch [3/20], Train Loss: 0.2821, Val Loss: 0.1816, Val Accuracy: 93.33%
Epoch [4/20], Train Loss: 0.2661, Val Loss: 0.1760, Val Accuracy: 93.33%
Epoch [5/20], Train Loss: 0.2617, Val Loss: 0.1728, Val Accuracy: 93.33%
Epoch [6/20], Train Loss: 0.2532, Val Loss: 0.1647, Val Accuracy: 93.33%
Epoch [7/20], Train Loss: 0.2428, Val Loss: 0.1588, Val Accuracy: 93.33%
Epoch [8/20], Train Loss: 0.2310, Val Loss: 0.1529, Val Accuracy: 93.33%
Epoch [9/20], Train Loss: 0.2308, Val Loss: 0.1474, Val Accuracy: 93.33%
Epoch [10/20], Train Loss: 0.2430, Val Loss: 0.1426, Val Accuracy: 96.67%
Epoch [11/20], Train Loss: 0.2228, Val Loss: 0.1395, Val Accuracy: 100.00%
Epoch [12/20], Train Loss: 0.2071, Val Loss: 0.1343, Val Accuracy: 100.00%
Epoch [13/20], Train Loss: 0.2074, Val Loss: 0.1285, Val Accuracy: 100.00%
Epoch [14/20], Train Loss: 0.2038, Val Loss:

In [22]:
# Evaluate the model on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Final Test Accuracy: {100 * correct / total:.2f}%')

Final Test Accuracy: 96.67%


In [25]:
print(predicted)

tensor([1, 1, 2, 1, 0, 2, 0, 0, 2, 0, 1, 2, 1, 0])


In [26]:
print(outputs)

tensor([[-3.5375,  2.0272,  1.3696],
        [-0.7054,  1.5739, -0.8543],
        [-3.0890,  0.7420,  2.1909],
        [-3.1081,  1.9119,  1.1253],
        [ 3.7698, -1.1075, -2.6729],
        [-3.9539,  0.2520,  3.5757],
        [ 4.0012, -1.2146, -2.7226],
        [ 3.6301, -1.1313, -2.5043],
        [-3.7948,  0.1218,  3.5714],
        [ 3.2924, -0.7268, -2.5243],
        [-1.8728,  1.1073,  0.7724],
        [-3.9521, -0.0632,  3.9356],
        [-1.7092,  2.0768, -0.3600],
        [ 4.4437, -1.5504, -2.8522]])
